# Программирование на Python 

# Семинар 7. Текстовые файлы, таблицы и основы работы в `pandas`

## Содержание

1. [Ключевое слово with и функция open()](#par1.1)
2. [Обработка информации из файла](#par1.2)
3. [Запись в файл](#par1.3)

<img src="images/sem_7_1.jpg" alt="Введение" width="400">

Для начала скачаем файлик [по ссылке](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/wine.csv). Для этого нам нужно нажать `Raw` и затем комбинацию `Ctrl+S` или `Cmd+S`.

[Источник данных - Kaggle](https://www.kaggle.com/datasets/jarredpriester/california-wine-production-19802020).

**Производство вин в Калифорнии 1980 - 2020 гг.**

Данные содержат следующие признаки:

* `Year` - Год
* `County` - Округ, где произвели вино
* `CountyCode` - Номер округа
* `HarvestedAcres` - Собранные акры винограда
* `Yield` - Урожайность на акр
* `Production` - Произведенная продукция (в тоннах)
* `Price` - Цена в далларах за тонну продукции

## Ключевое слово `with` и функция `open()` <a name="par1.1"></a>

Открыть текстовый файл (например, с расширением `.csv` или `.txt`) можно с помощью ключевого слова `with`. О чем тут важно помнить: 

+ файл мы кладем в ту же папку, где у нас лежит блокнот (можно и без этого, но тогда легко запутаться в пути к этому файлу на вашем компьютере)
+ название файла указываем полностью! 
+ указываем режим открытия файла
    + `'r'` = read/чтение (он стоит по умолчанию)
    + `'w'` = write/запись, создание
    + `'a'` = append/дозапись
    + на самом деле их больше, но это основные
   
### Открытие файла

Файл у нас большой, поэтому я попрошу напечатать не все строки из него, а только 5. 

In [1]:
line_num = 5 # задаем число строк

# открываем файл с помощью with()
with open('wine.csv', 'r', encoding = 'utf8') as inf:
    for line in inf:
        print(line)
        
        line_num -= 1
        if line_num == 0:
            break

Year,CountyCode,County,HarvestedAcres,Yield,Production,Price

2020,1,Alameda,2530.0,5.14,13000.0,1497.69

2020,5,Amador,5360.0,2.31,12400.0,1318.31

2020,9,Calaveras,579.0,3.06,1770.0,1325.99

2020,11,Colusa,747.0,6.02,4500.0,684.67



Самое важное, что тут нужно запомнить — в конце каждой строки (абзаца) стоит переход на новую строку `\n`. При обработке данных советую всегда его убирать, лишним не будет. Сделать это можно с помощью метода `.strip()`.

In [2]:
line_num = 5
with open('wine.csv', 'r', encoding = 'utf8') as inf:
    for line in inf:
        print(line.strip())
        
        line_num -= 1
        if line_num == 0:
            break

Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
2020,1,Alameda,2530.0,5.14,13000.0,1497.69
2020,5,Amador,5360.0,2.31,12400.0,1318.31
2020,9,Calaveras,579.0,3.06,1770.0,1325.99
2020,11,Colusa,747.0,6.02,4500.0,684.67


Обратите внимание - здесь и далее мы с вами проходимся по строкам в файле `.csv`. Что мы видим выше?

На первой строке находятся названия колонок:
```
Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
```

Когда мы смотрим на следующие строки, мы соотносим записанные значения с этой самой первой колонкой. Например, для первой строки с данными:

```
2020,1,Alameda,2530.0,5.14,13000.0,1497.69
```

Это означает, что в первой строке записаны следующие данные:
* `Year` - `2020`;
* `CountyCode` - `1`;
* `County` – `Alameda`;
* `HarvestedAcres` – `2530.0`;
* `Yield` – `5.14`;
* `Production` – `13000.0`;
* `Price` – `1497.69`.

Но как можно работать с данными, записанными в одной строке? Файл `.csv` расшировывается как `comma separated values` – то есть это тип файлов, в которых значения разделены запятыми. Это означает, что мы можем спокойно разделить наши значения по запятой и получить список.

In [3]:
# надо лишь применить метод .split(',')
'2020,1,Alameda,2530.0,5.14,13000.0,1497.69'.split(',')

['2020', '1', 'Alameda', '2530.0', '5.14', '13000.0', '1497.69']

Давайте придумаем нам содержательную задачку на этих данных. Например, посчитаем выручку округа (количество продукции в тоннах умножаем на цену за тонну) и посмотрим, какой округ заработал больше остальных в каждом году. 

Сохранять все будем в словарь `data`, где ключи — года, а значения — словари (с парами округ-выручка).

In [4]:
data = {} # создаем пустой словарь

# открываем файл
with open('wine.csv', 'r', encoding = 'utf8') as input_file:
    for line in input_file:
        line = line.strip().split(',') # превращаем строку в список
        
        if line[0].isdigit(): # если первое значение - это год,
            year = int(line[0]) # выделяем год
            county = line[2] # выделяем округ
            
            # получаем значение выручки
            if len(line[-2]) == 0 or len(line[-1]) == 0:
                revenue = 0
            else:
                revenue = float(line[-2]) * float(line[-1])
            
            # обновляем значение в словаре
            data.setdefault(year, {}) 
            data[year].update({county: revenue})        

## Обработка информации из файла <a name="par1.2"></a>

При работе с такими файлами главное — понять, как удобнее считать из него информацию и затем обработать ее. 

Вот так выглядит, например, информация за 2020 год:

In [5]:
# print(data)

print(data[2020])

{'Alameda': 19469970.0, 'Amador': 16347044.0, 'Calaveras': 2347002.3, 'Colusa': 3081015.0, 'ContraCosta': 6829044.3, 'ElDorado': 9663014.4, 'Fresno': 249876600.0, 'Kern': 59667600.00000001, 'Kings': 16954017.0, 'Lake': 52510114.99999999, 'Madera': 136102200.0, 'Marin': 937999.4, 'Mendocino': 82181760.0, 'Merced': 63419040.0, 'Monterey': 105991314.0, 'Napa': 461395463.99999994, 'Nevada': 0, 'Placer': 1964995.7099999997, 'Riverside': 18999981.0, 'Sacramento': 155975350.0, 'SanBenito': 34773072.0, 'SanBernardino': 440000.91, 'SanDiego': 5206013.9, 'SanJoaquin': 340911680.0, 'SanLuisObispo': 218238370.0, 'SanMateo': 0, 'SantaBarbara': 93836004.0, 'SantaClara': 11981970.8, 'SantaCruz': 4814000.0, 'Shasta': 500000.0, 'Solano': 21189056.0, 'Sonoma': 351511840.0, 'Stanislaus': 35557227.0, 'Tehama': 1053997.3599999999, 'Tulare': 35504000.0, 'Yolo': 104950500.0}


А так можно найти округ с максимальной выручкой:

In [6]:
print(sorted([(revenue, county) for county, revenue in data[2020].items()])[-1])

# привычный нам аналог:
stats = list()
for county, revenue in data[2020].items():
    stats.append((revenue, county))
print(sorted(stats)[-1])

(461395463.99999994, 'Napa')
(461395463.99999994, 'Napa')


Посмотрим на округа с максимальной выручкой по каждому году

In [7]:
counties = {}

for year in sorted(data):
    stats = list()
    for county, revenue in data[year].items():
        stats.append((revenue, county))
    res = sorted(stats)[-1]
    
    print(f'В {year} году больше всего выручки получил округ {res[1]}')
    print(f'Выручка составила {res[0]} доллара')

В 1980 году больше всего выручки получил округ Fresno
Выручка составила 62483400.0 доллара
В 1981 году больше всего выручки получил округ Fresno
Выручка составила 59860000.0 доллара
В 1982 году больше всего выручки получил округ Napa
Выручка составила 76285051.0 доллара
В 1983 году больше всего выручки получил округ Napa
Выручка составила 63605976.0 доллара
В 1984 году больше всего выручки получил округ Napa
Выручка составила 76796928.0 доллара
В 1985 году больше всего выручки получил округ Napa
Выручка составила 78286204.0 доллара
В 1986 году больше всего выручки получил округ Napa
Выручка составила 85707540.0 доллара
В 1987 году больше всего выручки получил округ Napa
Выручка составила 82147600.0 доллара
В 1988 году больше всего выручки получил округ SanJoaquin
Выручка составила 146273400.0 доллара
В 1989 году больше всего выручки получил округ Napa
Выручка составила 153706293.0 доллара
В 1990 году больше всего выручки получил округ Napa
Выручка составила 140479616.0 доллара
В 1991 г

## Запись информации в файл <a name="par1.3"></a>

А теперь попробуем все, что мы сейчас получили, записать файл. Для этого нам снова понадобится слово `with` и функция `open()`. Назовем наш файл `wine_result.txt`

In [8]:
with open('wine_result.txt', 'w', encoding = 'utf8') as output_file:
    # скопируем сюда предыдущий код
    for year in sorted(data):
        res = sorted([(revenue, county) for county, revenue in data[year].items()])[-1]
        
        print(f'В {year} году больше всего выручки получил округ {res[1]}', file = output_file)
        print(f'Выручка составила {res[0]} доллара', file = output_file)
    
    print('===', file = output_file)
    print('Подведем итог:', file = output_file)

Тут ячейка ничего не должна нам выдавать, однако(!) обратите внимание: теперь в папке с блокнотом появился файл `wine_result.txt` со всей информацией, которую мы только что туда записали. 

## Задачи: C-01

<img src="images/sem_7_2.jpg" alt="Практика" width="400">

<center><b><font size=4>Задача 1: C-01 </font></b></center>

Маша собирает информацию о фильмах, которые она посмотрела, и записывает их в файл. Напишите программу, которая выведет строки с информацией о фильмах определенного жанра.

**ФОРМАТ ВВОДА**  
- Дан структурированный текстовый файл **movies.csv**, в кодировке **utf-8**, где на каждой строке записана следующая информация о фильме: название, год выпуска, жанр, рейтинг. Например, **"Интерстеллар,2014,фантастика,8.6"**.
- Разделителями данных в файле служат запятые.
- Гарантируется, что в файле есть как минимум одна строка с данными.
- С клавиатуры вводится строка — жанр фильма.
- Гарантируется, что в файле есть как минимум один фильм этого жанра.

**ФОРМАТ ВЫВОДА**  
- Программа должна выводить строки из оригинального файла, которые соответствуют фильмам указанного жанра (не забудьте удалить символы конца строки!).

**ДОПОЛНИТЕЛЬНО**  
- Ваше решение будет проверяться на закрытых тестах. Ваш код должен считывать файл movies.csv без дополнительных указаний (например, пути к файлу на вашем компьютере).
- В открытом тесте используются файл [movies.csv](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/movies.csv). Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Ввод       | Результат | 
|-------|------------|-----------| 
| 0     | фантастика | Интерстеллар,2014,фантастика,8.6<br>Начало,2010,фантастика,8.8| 

In [9]:
# your code here (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧

<center><b><font size=4>Задача 2: C-01 </font></b></center>

Владелец магазина ведет учет продаж в текстовом файле. Напишите программу, которая выведет все продажи, сумма которых превышает 500 рублей.


**ФОРМАТ ВВОДА**  
- Дан структурированный текстовый файл **sales.txt**, в кодировке **utf-8**, где на каждой строке записана следующая информация о продаже: название товара, количество, сумма. Например, **"ноутбук,1,45000"**.
- Разделителями данных в файле служат запятые.
- Гарантируется, что в файле есть как минимум одна строка с данными.


**ФОРМАТ ВЫВОДА**  
- Программа должна выводить строки из оригинального файла, где сумма продажи превышает 500 рублей (не забудьте удалить символы конца строки!).

**ДОПОЛНИТЕЛЬНО**  
- Ваше решение будет проверяться на закрытых тестах. Ваш код должен считывать файл sales.txt без дополнительных указаний (например, пути к файлу на вашем компьютере).
- В открытом тесте используются файл [sales.txt](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/sales.txt). Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------|
| 0     | ноутбук,1,45000<br>мышь,2,1500<br>кружка,10,2000| 

In [10]:
# your code here (✿ヘᴥヘ)

<center><b><font size=4>Задача 3: C-01 </font></b></center>

Почтовое отделение ведет учет доставки посылок. Напишите программу, которая выведет информацию о посылках, вес которых превышает 2 кг.

**ФОРМАТ ВВОДА**  
- Дан структурированный текстовый файл **parcels.csv**, в кодировке **utf-8**, где на каждой строке записана следующая информация о посылке: город отправки, город назначения, вес (кг), стоимость доставки. Например, **"Москва,Казань,3,500"**.
- Разделителями данных в файле служат запятые.
- Гарантируется, что в файле есть как минимум одна строка с данными.

**ФОРМАТ ВЫВОДА**  
- Программа должна выводить строки из оригинального файла, где вес посылки превышает 2 кг (не забудьте удалить символы конца строки!).

**ДОПОЛНИТЕЛЬНО**  
- Ваше решение будет проверяться на закрытых тестах. Ваш код должен считывать файл parcels.csv без дополнительных указаний (например, пути к файлу на вашем компьютере).
- В открытом тесте используются файл [parsels.csv](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/parcels.csv). Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------|
| 0     | Москва,Казань,3,500<br>Екатеринбург,Краснодар,2.5,400| 

In [11]:
# your code here (✿˘ᴗ˘)♡

<center><b><font size=4>Задача 4: C-01 </font></b></center>

Компания по доставке еды ведет учет заказов. Напишите программу, которая выведет все заказы, которые нужно доставить в определенный район города.


**ФОРМАТ ВВОДА**  
- Дан структурированный текстовый файл **orders.csv**, в кодировке **utf-8**, где на каждой строке записана следующая информация о заказе: название блюда, стоимость, район доставки, время заказа. Например, **"пицца,700,Центр,18:30"**.
- Разделителями данных в файле служат запятые.
- Гарантируется, что в файле есть как минимум одна строка с данными. 
- С клавиатуры вводится строка — район доставки.
- Гарантируется, что в файле есть как минимум один заказ в этот район.


**ФОРМАТ ВЫВОДА**  
- Программа должна выводить строки из оригинального файла, которые соответствуют заказам в указанный район (не забудьте удалить символы конца строки!).


**ДОПОЛНИТЕЛЬНО**  
- Ваше решение будет проверяться на закрытых тестах. Ваш код должен считывать файл orders.csv без дополнительных указаний (например, пути к файлу на вашем компьютере).
- В открытом тесте используются файл [orders.csv](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/orders.csv). Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Ввод       | Результат | 
|-------|------------|-----------| 
| 0     | Центр      | пицца,700,Центр,18:30| 

In [12]:
# your code here (◍•ᴗ•◍)

## Задачи: C-02

<center><b><font size=4>Задача 1: C-02 </font></b></center>

Алексей изучает маршрут путешественника и хочет узнать, сколько раз каждый город был посещен. Названия городов начинаются с большой буквы.

**ФОРМАТ ВВОДА**  
- Текстовый файл **route.txt**, в котором записано описание маршрута. Названия городов начинаются с большой буквы.
- Гарантируется, что других слов, начинающихся с большой буквы, в файле нет.
- Также гарантируется, что до и после названия города стоят пробелы.


**ФОРМАТ ВЫВОДА**  
- Строка — название города, который посещался чаще других.
- Гарантируется, что такой город только один.

**ДОПОЛНИТЕЛЬНО**  
- В открытом тесте используются файл [route.txt](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/route.txt)  Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------|
| 0     | Москва    | 

In [13]:
# your code here (｡♡‿♡｡)

<center><b><font size=4>Задача 2: C-02 </font></b></center>

Марина изучает конспект по биологии и помечает непонятные слова знаком \*. Помогите Марине собрать все непонятные слова длиной более 5 символов и подсчитайте, сколько раз каждое слово было помечено.


**ФОРМАТ ВВОДА**  
- Текстовый файл **biology.txt**, в котором записан конспект Марины с пометками. Непонятым считается любое слово, заканчивающееся знаком \*. Между словом и \* точно не стоят пробелы.

**ФОРМАТ ВЫВОДА**  
- Словарь, где ключи — непонятные слова длиной более 5 символов (без \*), а значения — количество раз, которое каждое слово было помечено. Слова должны быть добавлены в словарь в том же порядке, в каком они встречались в оригинальном файле.
- На следующей строке выведите первое по алфавиту слово из словаря.

**ДОПОЛНИТЕЛЬНО**  
- В открытом тесте используются файл [biology.txt](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/biology.txt). Вы можете скачать его и проверить свое решение локально.
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------|
| 0     | { 'биология': 4, 'клетка': 3, 'растение': 3, 'фотосинтез': 1, 'хлоропласт': 2, 'митохондрия': 2, 'сердце': 2, 'животное': 1}<br>биология| 

In [14]:
# your code here (◕ᴥ◕ʋ)

<center><b><font size=4>Задача 3: C-01 </font></b></center>

Иван изучает, какие тарифы мобильной связи приносят больший доход компании. Помогите Ивану собрать информацию о среднем доходе по каждому тарифу и определить самый прибыльный тариф.

**ФОРМАТ ВВОДА**  
- Структурированный файл **tariffs.csv**, в котором записана информация о клиентах и выбранном тарифе. В каждой строке записан ID клиента, название тарифа и его стоимость через точку с запятой. Например, ID1234;Базовый;300.
- Гарантируется, что в ID клиентов и в названиях тарифов нет ";" (точки с запятой). 
- В каждой строке содержится информация только об одном клиенте.

**ФОРМАТ ВЫВОДА**  
- Строка — название тарифа с самым высоким средним доходом.
- Гарантируется, что такой тариф только один.

**ДОПОЛНИТЕЛЬНО**  
- В открытом тесте используются файл [tariffs.csv](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/tariffs.csv). Вы можете скачать его и проверить свое решение локально;
- в примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------|
| 0     | Премиум   | 

In [15]:
# your code here (｡♥‿♥｡)⊃━☆ﾟ.*

<center><b><font size=4>Задача 4: C-01 </font></b></center>

Ольга изучает, какие десерты представлены в кафе ее города. Помогите Ольге собрать информацию о средних ценах на десерты и определить самый дорогой десерт.


**ФОРМАТ ВВОДА**  
- Структурированный файл **desserts.csv**, в котором через точку с запятой записана информация о десертах, их цене и кафе.
- Каждая строка имеет вид: <название десерта>;<цена>;<название кафе>. Например, Чизкейк;250;Кофейня №1.
- Гарантируется, что в названиях десертов и кафе нет ";" (точки с запятой). 
- В каждой строке записана информация только об одном десерте.

**ФОРМАТ ВЫВОДА**  
- Строка — название в среднем самого дорогого десерта.
- Гарантируется, что такой десерт только один.


**ДОПОЛНИТЕЛЬНО**  
- В открытом тесте используются файл [desserts.csv](https://github.com/nevolinaa/hse_py_seminars_25/blob/main/data/desserts.csv). Вы можете скачать его и проверить свое решение локально. 
- В примере в первой колонке отображается номер теста (0 — открытый тест), считывать это число или использовать его в вашем решении не нужно.

| Тест	| Результат | 
|-------|-----------| 
| 0     | Тирамису  | 

In [16]:
# your code here (✧ω✧)ノ～♡